# Modelo Predictivo

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

# ── 1) Datos de ejemplo ──────────────────────────────────────────────
df = pd.DataFrame({
    "antiguedad_anos":  [1, 3, 8, 2, 5, 4, 10, 7],
    "satisfaccion":     [0.4, 0.9, 0.2, 0.7, 0.8, 0.6, 0.3, 0.5],
    "salario_nivel":    [1, 2, 3, 2, 2, 2, 3, 1],      # 1-bajo / 3-alto
    "promociones":      [0, 1, 2, 0, 1, 1, 3, 2],
    "renuncia":         [1, 0, 1, 0, 0, 0, 1, 1]       # 1=se fue, 0=se quedó
})

X = df.drop("renuncia", axis=1)
y = df["renuncia"]

# ── 2) Split y escalado ───────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# ── 3) Entrenar modelo ────────────────────────────────────────────────
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

# ── 4) Evaluar ────────────────────────────────────────────────────────
y_pred = logreg.predict(X_test_scaled)
y_prob = logreg.predict_proba(X_test_scaled)[:, 1]
print("AUC:", roc_auc_score(y_test, y_prob))
print(classification_report(y_test, y_pred))

# ── 5) Interpretación rápida: coeficientes ────────────────────────────
for name, coef in zip(X.columns, logreg.coef_[0]):
    print(f"{name:15s}  β = {coef:+.3f}")


AUC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

antiguedad_anos  β = +0.299
satisfaccion     β = -1.011
salario_nivel    β = -0.516
promociones      β = +0.261


P(Renuncia) = 1 / (1 + e^-(β0 + β1*Años + β2*Salario + β3*Satisfacción + ...))


# Machine Learning

# Análisis Paso a Paso del Modelo de Predicción de Renuncia
Este código implementa un modelo de Random Forest para predecir la renuncia de empleados basado en varias características. Vamos a desglosarlo:

1) Preparación de Datos
Se crea un DataFrame con 8 empleados y 9 variables:

Variables predictoras: antigüedad, satisfacción, nivel salarial, promociones, horas extra, ausencias, distancia al trabajo, formación

Variable objetivo: renuncia (1=Sí, 0=No)

2) División de Datos
Se separa en conjuntos de entrenamiento (75%) y prueba (25%)

stratify=y asegura proporción similar de renuncias en ambos conjuntos

Resultado: 6 muestras entrenamiento, 2 muestras prueba

3) Entrenamiento con Grid Search
Se usa Random Forest Classifier

* Búsqueda de hiperparámetros con GridSearchCV:

    * Prueba 200 y 400 árboles (n_estimators)

    * Prueba profundidades máxima: None (sin límite), 4 y 6 niveles

    1. ¿Qué es Grid Search?
        Grid Search es una técnica de optimización de hiperparámetros que prueba todas las combinaciones posibles de valores dados para encontrar la mejor configuración del modelo.

        Hiperparámetros: Son configuraciones del modelo que no se aprenden de los datos, sino que se definen antes del entrenamiento (ej: número de árboles, profundidad máxima).

        GridSearchCV prueba todas las combinaciones posibles usando validación cruzada (CV) para evitar sobreajuste.
    
    2. Configuración del Random Forest
        El modelo base es un RandomForestClassifier, que funciona creando múltiples árboles de decisión y combinando sus resultados para mejorar la precisión y reducir el sobreajuste.

        Hiperparámetros Optimizados en este Ejemplo:
        
        * n_estimators: Número de árboles en el bosque.

            * Valores probados: [200, 400] (se prueban bosques con 200 y 400 árboles).

            * Más árboles mejoran la estabilidad, pero aumentan el costo computacional.

        * max_depth: Profundidad máxima de cada árbol.

            * Valores probados: [None, 4, 6]

            * None: Sin límite (los árboles crecen hasta que todas las hojas son puras o hasta min_samples_split).

            * 4 y 6: Limita la profundidad para evitar sobreajuste (árboles más simples).
    3. ¿Cómo funciona GridSearchCV?
        El proceso es el siguiente:

        1. Define el espacio de búsqueda (todas las combinaciones posibles):

            * n_estimators=200, max_depth=None

            * n_estimators=200, max_depth=4

            * n_estimators=200, max_depth=6

            * n_estimators=400, max_depth=None

            * n_estimators=400, max_depth=4

            * n_estimators=400, max_depth=6
            → Total: 6 combinaciones.

        2. Validación Cruzada (CV=3):

            * Divide los datos de entrenamiento en 3 partes (folds).

            * Entrena el modelo en 2 folds y evalúa en 1 fold, rotando hasta probar todas las combinaciones.
            
            * Repite esto para cada combinación de hiperparámetros.

        3. Selección del mejor modelo:

            * Usa AUC (Area Under the ROC Curve) como métrica de evaluación (ideal para problemas de clasificación desbalanceada).

            * Elige la combinación que maximiza el AUC promedio en validación cruzada.
        4. Resultados del Grid Search
            En este caso, el mejor modelo fue uno de los probados (no sabemos cuál exactamente sin ver grid.best_params_), pero sabemos que:

            * AUC = 1.0 (rendimiento perfecto en los datos de prueba).

            * Esto sugiere que alguna combinación de n_estimators y max_depth logró un ajuste perfecto (aunque podría ser sobreajuste debido al pequeño dataset).

        5. ¿Por qué usar Grid Search?
            * Evita el "ensayo y error" manual de hiperparámetros.

            * Optimiza automáticamente el modelo para la métrica deseada (en este caso, AUC).

            * Reduce el riesgo de sobreajuste al usar validación cruzada.
        ➡️ Posibles Mejoras:
            * Probar más valores (ej: n_estimators=[50, 100, 200, 400]).

            * Incluir otros hiperparámetros como min_samples_split, max_features, etc.

            * Usar RandomizedSearchCV si el espacio de búsqueda es muy grande (muestra combinaciones aleatorias en lugar de todas).

        ✅ Conclusión
        El Grid Search con Random Forest permite encontrar la mejor combinación de hiperparámetros de manera sistemática, mejorando el rendimiento del modelo. En este caso, el resultado fue perfecto (AUC=1.0), pero en un escenario real con más datos, el proceso ayudaría a equilibrar precisión y generalización.

Se selecciona el mejor modelo según AUC (Area Under Curve)

4) Evaluación del Modelo
Resultados Obtenidos:
AUC: 1.0 (Perfecto, máxima capacidad de discriminación)

Classification Report:

Precision, Recall y F1-score todos en 1.0 (perfectos)

El modelo acertó en ambas predicciones de prueba

Interpretación:
El modelo tuvo rendimiento perfecto en este pequeño conjunto de prueba

Sin embargo, con solo 2 muestras de prueba, estos resultados no son estadísticamente significativos

En un escenario real necesitaríamos más datos para evaluación confiable

5) Importancia de Variables
El código calcula pero no muestra los valores específicos de importancia

Estas mostrarían qué variables contribuyeron más a las predicciones (ej: satisfacción, antigüedad)

Limitaciones a Considerar:
Tamaño de muestra extremadamente pequeño (8 registros)

Evaluación en solo 2 muestras de prueba

Posible sobreajuste (overfitting) debido a lo anterior

Resultados perfectos suelen ser sospechosos en casos reales

Recomendaciones para Mejora:
Obtener más datos (cientos o miles de registros)

Validación cruzada más robusta

Considerar otras métricas además de AUC

Explorar balance de clases (¿proporción de renuncias?)

En resumen, este es un ejemplo didáctico que muestra el flujo completo de un modelo de machine learning, pero los resultados perfectos reflejan las limitaciones del pequeño conjunto de datos más que un rendimiento realista.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

# ── 1) Datos con más riqueza ──────────────────────────────────────────
df = pd.DataFrame({
    "antiguedad_anos":      [1, 3, 8, 2, 5, 4, 10, 7],
    "satisfaccion":         [0.4, 0.9, 0.2, 0.7, 0.8, 0.6, 0.3, 0.5],
    "salario_nivel":        [1, 2, 3, 2, 2, 2, 3, 1],
    "promociones":          [0, 1, 2, 0, 1, 1, 3, 2],
    "horas_extra_mes":      [10, 2, 5, 8, 1, 0, 12, 9],
    "dias_ausencia":        [2, 0, 4, 1, 0, 0, 5, 3],
    "distancia_km":         [8, 15, 2, 25, 12, 9, 3, 30],
    "formacion_cursos":     [3, 5, 1, 4, 6, 2, 0, 2],
    "renuncia":             [1, 0, 1, 0, 0, 0, 1, 1]
})

X = df.drop("renuncia", axis=1)
y = df["renuncia"]

# ── 2) Split ──────────────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    random_state=42, stratify=y)

# ── 3) Entrenar con búsqueda de hiperparámetros ───────────────────────
rf = RandomForestClassifier(random_state=42)
param_grid = {"n_estimators": [200, 400],
              "max_depth": [None, 4, 6]}
grid = GridSearchCV(rf, param_grid, cv=3, scoring="roc_auc")
grid.fit(X_train, y_train)

best_rf = grid.best_estimator_

# ── 4) Evaluar ────────────────────────────────────────────────────────
y_pred = best_rf.predict(X_test)
y_prob = best_rf.predict_proba(X_test)[:, 1]
print("AUC:", roc_auc_score(y_test, y_prob))
print(classification_report(y_test, y_pred))

# ── 5) Importancia de variables ───────────────────────────────────────
importances = pd.Series(best_rf.feature_importances_, index=X.columns).sort_values(ascending=False)
from IPython.display import display, Markdown

display(Markdown("### 📊 Importancia de Variables"))
display(importances)

display(Markdown("### 📈 Métricas de Evaluación"))
display(Markdown(f"**AUC:** `{roc_auc_score(y_test, y_prob):.4f}`"))
display(Markdown("**Classification Report:**"))
print(classification_report(y_test, y_pred))  # Se imprime normal


AUC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



### 📊 Importancia de Variables

dias_ausencia       0.225694
satisfaccion        0.206250
horas_extra_mes     0.131641
salario_nivel       0.108854
distancia_km        0.106597
antiguedad_anos     0.080990
formacion_cursos    0.072786
promociones         0.067188
dtype: float64

### 📈 Métricas de Evaluación

**AUC:** `1.0000`

**Classification Report:**

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



# Interpretación Detallada de los Resultados del Modelo

## Análisis de la Importancia de Variables

El modelo de Random Forest ha asignado las siguientes importancias a cada variable predictora:

| Variable           | Importancia | Interpretación |
|--------------------|-------------|----------------|
| dias_ausencia      | 0.226       | La variable más importante, sugiere que las ausencias laborales son el principal predictor de renuncia. |
| satisfaccion       | 0.206       | El segundo factor más relevante, indica que empleados menos satisfechos tienen mayor probabilidad de renunciar |
| horas_extra_mes    | 0.132       | Las horas extras muestran un impacto moderado en la decisión de renuncia. |
| salario_nivel      | 0.109       | El nivel salarial tiene una influencia perceptible pero menor que los factores anteriores. |
| distancia_km       | 0.107       | La distancia al trabajo aparece como un factor relevante pero no determinante. |
| antiguedad_anos    | 0.081       | La antigüedad en la empresa tiene un impacto relativamente bajo. |
| formacion_cursos   | 0.073       | La formación recibida muestra una influencia menor |
| promociones        | 0.067       | Las promociones aparecen como el factor menos influyente. |

**Interpretación clave**: Las políticas para reducir ausencias y mejorar la satisfacción laboral serían las más efectivas para retener empleados.

## Evaluación del Rendimiento del Modelo

### Métrica AUC (Area Under Curve)
- **AUC = 1.0** (perfecto)
- **Significado**: Clasificación correcta del 100% de casos
- **Contexto**: Con solo 2 muestras de prueba, interpretar con cautela

### Classification Report

| Métrica       | Clase 0 (No renuncia) | Clase 1 (Renuncia) |
|---------------|----------------------|-------------------|
| Precision     | 1.00                 | 1.00              |
| Recall        | 1.00                 | 1.00              |
| F1-score      | 1.00                 | 1.00              |
| Support       | 1 caso               | 1 caso            |

**Interpretación**:
- Todas las métricas muestran rendimiento perfecto
- Accuracy del 100% en el conjunto de prueba

## Limitaciones y Consideraciones

1. **Tamaño de muestra pequeño**:
   - 8 casos entrenamiento, 2 pruebas
   - Riesgo de sobreajuste (overfitting)

2. **Validación adicional necesaria**:
   - Requiere más datos para confirmación
   - Validación cruzada recomendable

3. **Contexto empresarial**:
   - Correlación ≠ causalidad
   - Posibles variables omitidas

## Recomendaciones de Acción

```python
# Pseudocódigo para implementación práctica
def recomendaciones_HR():
    acciones = [
        "Recolectar más datos (100+ empleados)",
        "Implementar programas anti-ausentismo",
        "Mejorar satisfacción laboral",
        "Revisar políticas de horas extras",
        "Monitoreo continuo con nuevos datos",
        "Validar con estudios cualitativos"
    ]
    return acciones